In [25]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import os
import glob
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import  applications
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D,GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

import warnings
warnings.filterwarnings('ignore')
import random
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
import glob
import shutil
import os
import itertools
from sklearn.metrics import confusion_matrix ,classification_report
import joblib

In [12]:
train_path = "/home/bee110493/.config/JetBrains/DataSpell2021.3/projects/meal-recommender/app/fruits/train"
test_path = "/home/bee110493/.config/JetBrains/DataSpell2021.3/projects/meal-recommender/app/fruits/test"

In [13]:
IMG_SIZE = (100, 100)
train_images=tf.keras.preprocessing.image_dataset_from_directory(directory=train_path,image_size=IMG_SIZE,batch_size=10,shuffle=False)
test_images= tf.keras.preprocessing.image_dataset_from_directory(directory=test_path, image_size=IMG_SIZE,batch_size=10,shuffle=False)

Found 460 files belonging to 3 classes.
Found 130 files belonging to 3 classes.


In [14]:
model1 = Sequential([

    Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same',input_shape=(100,100,3)),
    MaxPool2D(pool_size=(2,2),strides=2),
    Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same'),
    MaxPool2D(pool_size=(2,2),strides=2),

    Flatten(),
    Dense(units=131,activation='softmax'),
])

In [15]:
model1.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 100, 100, 32)      896       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 50, 50, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 50, 50, 32)        9248      
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 25, 25, 32)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 20000)             0         
                                                                 
 dense_3 (Dense)             (None, 131)              

In [16]:
model1.compile(optimizer=Adam(learning_rate=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [17]:
results=model1.fit(train_images,epochs=10)

Epoch 1/10
46/46 [==============================] - 6s 111ms/step - loss: 82.6405 - accuracy: 0.6652
Epoch 2/10
46/46 [==============================] - 6s 123ms/step - loss: 175.1866 - accuracy: 0.1109
Epoch 3/10
46/46 [==============================] - 6s 138ms/step - loss: 15.9251 - accuracy: 0.3543
Epoch 4/10
46/46 [==============================] - 5s 118ms/step - loss: 8.6937 - accuracy: 0.3935
Epoch 5/10
46/46 [==============================] - 6s 125ms/step - loss: 5.1094 - accuracy: 0.5022
Epoch 6/10
46/46 [==============================] - 6s 124ms/step - loss: 3.0788 - accuracy: 0.5826
Epoch 7/10
46/46 [==============================] - 6s 131ms/step - loss: 2.2182 - accuracy: 0.6261
Epoch 8/10
46/46 [==============================] - 5s 117ms/step - loss: 1.9654 - accuracy: 0.6435
Epoch 9/10
46/46 [==============================] - 5s 117ms/step - loss: 1.7037 - accuracy: 0.6804
Epoch 10/10
46/46 [==============================] - 6s 127ms/step - loss: 1.4673 - accuracy: 0.

In [20]:
test_model, test_acc = model1.evaluate(test_images, verbose=2)
print('\nTest accuracy:', test_acc)

13/13 - 1s - loss: 2.5483 - accuracy: 0.7308 - 793ms/epoch - 61ms/step

Test accuracy: 0.7307692170143127


In [21]:
probability_model = tf.keras.Sequential([model1,
                                         tf.keras.layers.Softmax()])

In [22]:
predictions = probability_model.predict(test_images)

13/13 [==============================] - 1s 55ms/step


In [26]:
joblib.dump(model1, 'model1.pkl')

['model1.pkl']

In [28]:
!ls

app		      first.ipynb  multi-page.py  num	     uber.py
classification.ipynb  model1.pkl   new.py	  README.md
